In [2]:
import pandas as pd

In [8]:
from neefasa_proj3 import *
import json
import requests
import datetime as dt

import os
from dotenv import load_dotenv

# Load CSVs
The flight information that I used was from bts.gov

## Load Flight data
I focused on flight from O'hare during Feb 2019

In [16]:
flight_data = pd.read_csv('438043864_T_ONTIME_REPORTING.csv')

## Other tables that  will be uploaded to AWS

In [17]:
cancel_code = pd.read_csv('L_CANCELLATION.csv_')

In [4]:
planes = pd.read_csv('ReleasableAircraft.2019/MASTER.txt')

In [18]:
models = pd.read_csv('ReleasableAircraft.2019/ACFTREF.txt')

In [19]:
carriers = pd.read_csv('L_UNIQUE_CARRIERS.csv_')

# Upload to postgres in AWS

## Load params

In [10]:
load_dotenv()

True

In [11]:
params = {
  'user': 'ubuntu',
  'port': 5432,
  'dbname': 'proj3'
}

In [12]:
params['host'] = os.getenv('psql_host')

In [13]:
params['password'] = os.getenv('psql_pwd')

## Upload tables
The function I'm using can be found in my module.

In [21]:
aws_psql(flight_data,'flights', params)

In [14]:
aws_psql(planes,'planes', params)

In [20]:
aws_psql(models,'models', params)

In [41]:
aws_psql(carriers,'carriers', params)

In [23]:
aws_psql(cancel_code,'cancelcode', params)

# Load 2018 
Desiring to do some feature engineering I downloaded the flight info for all of 2018. I used this to find the ontime rate for different features. 

In [29]:
flight2018 = pd.read_csv('2018flights/Jan2018flights.csv')
aws_psql(flight2018,'flights2018',params)

The rest of the months I uploaded and copied to psql in a term.

In [30]:
!ls 2018flights

Apr2018flights.csv Feb2018flights.csv Jun2018flights.csv Nov2018flights.csv
Aug2018flights.csv Jan2018flights.csv Mar2018flights.csv Oct2018flights.csv
Dec2018flights.csv Jul2018flights.csv May2018flights.csv Sep2018flights.csv


In [31]:
months = ['Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [34]:
for month in months:
    fname = f'2018flights/{month}2018flights.csv'
    onemonth = pd.read_csv(fname)
    aws_psql(onemonth,'flights2018',params, new_table=False)

## Find average delay for flights in 2018

### Delay Rate for Airlines

In [42]:
q1 = '''select "OP_UNIQUE_CARRIER", count("DEP_DEL15") as DEL_FLIGHTS
from flights2018
where "DEP_DEL15" > 0 and "ORIGIN" = 'ORD'
group by "OP_UNIQUE_CARRIER";'''

In [36]:
q2 = '''select "OP_UNIQUE_CARRIER", count("OP_UNIQUE_CARRIER" ) as TOTAL_FLIGHTS
from flights2018
where "ORIGIN"='ORD'
group by "OP_UNIQUE_CARRIER";'''

In [37]:
q3 = '''select "OP_UNIQUE_CARRIER", count("CANCELLED") 
from flights2018
where "CANCELLED" =1  and "ORIGIN"='ORD'
group by "OP_UNIQUE_CARRIER";'''

Carrier total delays

In [47]:
Airline_delays = psql_to_df(q1, params, col=['Airline','Total_Delays'])

Carrier total flights

In [49]:
Airline_delays = Airline_delays.merge(psql_to_df(q2, params, col=['Airline','Total_Flights']), on='Airline')

Carrier total Cancelations

In [50]:
Airline_delays = Airline_delays.merge(psql_to_df(q3, params, col=['Airline','Can_Flights']), on='Airline')

Calculate Rates

In [51]:
Airline_delays['carrier_delay_rate'] = Airline_delays['Total_Delays'] / Airline_delays['Total_Flights']

In [52]:
Airline_delays['carrier_cancel_rate'] = Airline_delays['Can_Flights'] / Airline_delays['Total_Flights']

In [53]:
Airline_delays['carrier_total_rate'] = Airline_delays['carrier_cancel_rate'] + Airline_delays['carrier_delay_rate']

In [54]:
Airline_delays = Airline_delays.set_index('Airline')

In [55]:
Airline_delays.to_csv('new_features/carrier_rates.csv')

### Flight number delay rate

In [60]:
Tot_flnum ='''select "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM" , count("OP_CARRIER_FL_NUM" ) 
from flights2018
where "ORIGIN"='ORD'
group by "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM"
order by "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM";'''

In [61]:
Del_flnum = '''select "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM" , count("DEP_DEL15") 
from flights2018
where "DEP_DEL15" =1  and "ORIGIN"='ORD'
group by "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM"
order by "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM";'''

In [72]:
can_flnum = '''select "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM" , count("CANCELLED") 
from flights2018
where "CANCELLED" =1  and "ORIGIN"='ORD'
group by "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM"
order by "OP_UNIQUE_CARRIER", "OP_CARRIER_FL_NUM";'''

In [77]:
flnum_tot = psql_to_df(Tot_flnum,col=['carrier','flnum','tot'])
flnum_del = psql_to_df(Del_flnum,col=['carrier','flnum','del'])
flnum_can = psql_to_df(can_flnum,col=['carrier','flnum','can'])

In [89]:
flnum_rates = flnum_tot.merge(flnum_del, on=['carrier','flnum']).merge(flnum_can, on=['carrier','flnum'])

In [95]:
flnum_rates['flnum_delay_rate'] = flnum_rates['del'] / flnum_rates['tot']

In [96]:
flnum_rates['flnum_cancel_rate'] = flnum_rates['can'] / flnum_rates['tot']

In [97]:
flnum_rates['flnum_total_rate'] = flnum_rates['flnum_cancel_rate'] + flnum_rates['flnum_delay_rate']

In [102]:
flnum_rates['Car_flnum'] = flnum_rates['carrier'].astype(str) + flnum_rates['flnum'].astype(str)

In [104]:
flnum_rates.to_csv('new_features/flnum_rates.csv')

### Tail number delay rate

In [129]:
tot_tail = '''select "TAIL_NUM" , count(*) 
from flights2018
where "ORIGIN"='ORD'
group by "TAIL_NUM" 
order by "TAIL_NUM" ;'''

In [130]:
del_tail = '''select "TAIL_NUM" , count("DEP_DEL15") 
from flights2018
where "DEP_DEL15" =1  and "ORIGIN"='ORD'
group by "TAIL_NUM" 
order by "TAIL_NUM" ;'''

In [131]:
can_tail = '''select "TAIL_NUM" , count("CANCELLED" ) 
from flights2018
where "CANCELLED" =1  and "ORIGIN"='ORD'
group by "TAIL_NUM" 
order by "TAIL_NUM" ;'''

In [132]:
tail_tot = psql_to_df(tot_tail,col=['tail_num','tot'])
tail_del = psql_to_df(del_tail,col=['tail_num','del'])
tail_can = psql_to_df(can_tail,col=['tail_num','can'])

In [133]:
tail_num_rates = tail_tot.merge(tail_del, on=['tail_num']).merge(tail_can, on=['tail_num'])

In [135]:
tail_num_rates['tailnum_delay_rate'] = tail_num_rates['del'] / tail_num_rates['tot']

In [136]:
tail_num_rates['tailnum_cancel_rate'] = tail_num_rates['can'] / tail_num_rates['tot']

In [137]:
tail_num_rates['tailnum_total_rate'] = tail_num_rates['tailnum_cancel_rate'] + tail_num_rates['tailnum_delay_rate']

In [147]:
tail_num_rates.to_csv('new_features/tailnum_rates.csv')

# Weather

In [56]:
ORDcoord = '41.9760648,-87.9067206'

In [57]:
darkskykey = os.getenv('darkskykey')

In [58]:
feb2019 = pd.date_range(start='2/1/2019',end='2/28/2019')

In [59]:
febweather = []

In [60]:
for date in feb2019:
    day = date.strftime('%Y-%m-%d')
    time = day+'T00:00:00'
    darkskyurl = 'https://api.darksky.net/forecast/'+darkskykey+'/'+ORDcoord+','+time
    febweather.append(requests.get(darkskyurl).json())

## daily weather

In [61]:
dailyfebweather = []
for day in febweather:
    dailyfebweather.append(day['daily']['data'][0])

In [62]:
febW = pd.DataFrame(dailyfebweather)

In [63]:
febW = febW.set_index(feb2019)

In [64]:
febW.to_csv('new_features/febW.csv')

## Hourly weather

In [65]:
hourlyfebweather = []
for day in febweather:
    hourlyfebweather.extend(day['hourly']['data'])

In [66]:
febWhourly = pd.DataFrame(hourlyfebweather)

In [67]:
febWhourly.to_csv('new_features/febWhourly.csv')